In [1]:
from gpt.tokenizer import train
import json
import base64
from config import DATA_DIR, EOT


print("Imports and global definitions loaded.")
print(f"Data directory: {DATA_DIR}")

Imports and global definitions loaded.
Data directory: /Users/jihonggan/code/gpt/data


In [2]:
def train_tinystories_tok():
    vocab, merges = train(
        DATA_DIR / "TinyStoriesV2-GPT4-train.txt",
        10000,
        [EOT],
    )

    out = DATA_DIR / "tinystories_bpe"
    out.mkdir(exist_ok=True)

    # ── vocab: id → bytes (stored as base64)
    with open(out / "vocab.json", "w") as f:
        json.dump(
            {
                str(i): base64.b64encode(tok).decode("ascii")
                for i, tok in vocab.items()
            },
            f,
        )

    # ── merges: one "token1 token2" per line, as base64
    with open(out / "merges.txt", "w") as f:
        for a, b in merges:
            a_str = base64.b64encode(a).decode("ascii")
            b_str = base64.b64encode(b).decode("ascii")
            f.write(f"{a_str} {b_str}\n")

train_tinystories_tok()

print("Tokenizer training complete.")
print(f"Out directory: {DATA_DIR / 'tinystories_bpe'}")

Tokenizer training complete.
Out directory: /Users/jihonggan/code/gpt/data/tinystories_bpe
